In [1]:
from abc import ABC, abstractmethod

## FACTORY METHOD

In [2]:
class Proxy(ABC):

    def __init__(self, timeout: int):
        self.timeout = timeout

    @abstractmethod
    def execute_request(self) -> bool:
        pass

class LumProxy(Proxy):

    def __init__(self):
        super().__init__(timeout=10)

    def execute_request(self) -> bool:
        print('Realizando peticion a lum.com ...')
        print('El servicio esperara hasta {} segundos'.format(self.timeout))
        return True
    
class BeeProxy(Proxy):

    def __init__(self):
        super().__init__(timeout=8)

    def execute_request(self) -> bool:
        print('Realizando peticion a bee.com ...')
        print('El servicio esperara hasta {} segundos'.format(self.timeout))
        return True

In [3]:
#SIN FABRICA
t = LumProxy()
t.execute_request()

t = BeeProxy()
t.execute_request()

Realizando peticion a lum.com ...
El servicio esperara hasta 10 segundos
Realizando peticion a bee.com ...
El servicio esperara hasta 8 segundos


True

In [40]:
class ProxyCreator(ABC):

    @abstractmethod
    def create_proxy(self):
        pass

    def launch_scrape(self) -> bool:

        # Call the factory method to create a Proxy object.
        proxy = self.create_proxy()

        print('Ejecutando peticion ...')
        result = proxy.execute_request()

        return result
    
class LumProxyCreator(ProxyCreator):
    def create_proxy(self) -> Proxy:
        return LumProxy()


class BeeProxyCreator(ProxyCreator):
    def create_proxy(self) -> Proxy:
        return BeeProxy()

In [41]:
#CON FABRICA
print('Arrancando con Lum ...')
LumProxyCreator().launch_scrape()

print('Arrancando con Bee ...')
BeeProxyCreator().launch_scrape()

Arrancando con Lum ...
Ejecutando peticion ...
Realizando peticion a lum.com ...
El servicio esperara hasta 10 segundos
Arrancando con Bee ...
Ejecutando peticion ...
Realizando peticion a bee.com ...
El servicio esperara hasta 8 segundos


True

## ABSTRACT FACTORY

In [24]:
#CLASES ABSTRACTAS PARA LOS PRODUCTOS
class AbstractStorage(ABC):

    def __init__(self, bucket):
        self.bucket = bucket

    @abstractmethod
    def create_bucket(self):
        pass

    @abstractmethod
    def export_to_file(self, path, content):
        pass

class AbstractDatabase(ABC):

    def __init__(self, db_name, table_name):
        self.db_name = db_name
        self.table_name = table_name

    @abstractmethod
    def connect(self):
        pass

    @abstractmethod
    def insert_record(self, record):
        pass

#CLASES CONCRETAS PARA LOS PRODUCTOS DE AWS
class AWSStorage(AbstractStorage):

    def __init__(self, bucket, max_retention):
        self.max_retention = max_retention
        super().__init__(bucket)

    def create_bucket(self):
        print('Creando bucket {} con retencion de {} dias en **S3**'.format(self.bucket, self.max_retention))

    def export_to_file(self, path, content):
        print('Exportando a archivo {} en **S3** el contenido: {}'.format(path, content))


class AWSDatabase(AbstractDatabase):

    def __init__(self, db_name, table_name):
        super().__init__(db_name, table_name)

    def connect(self):
        print('Conectando a la base de datos {} en **RDS**'.format(self.db_name))

    def insert_record(self, record):
        print('Insertando el registro {} en la tabla {} de **RDS**'.format(record, self.table_name))


#CLASES CONCRETAS PARA LOS PRODUCTOS DE AZURE
class AzureStorage(AbstractStorage):

    def __init__(self, bucket, is_encrypted):
        self.is_encrypted = is_encrypted
        super().__init__(bucket)

    def create_bucket(self):
        print('Creando bucket {} con encriptacion {} en **Blobstorage**'.format(self.bucket, self.is_encrypted))

    def export_to_file(self, path, content):
        print('Exportando a archivo {} en **Blobstorage** el contenido: {}'.format(path, content))


class AzureDatabase(AbstractDatabase):

    def __init__(self, db_name, table_name):
        super().__init__(db_name, table_name)

    def connect(self):
        print('Conectando a la base de datos {} en **Azure**'.format(self.db_name))

    def insert_record(self, record):
        print('Insertando el registro {} en la tabla {} de **Azure**'.format(record, self.table_name))

In [28]:
#SIN FABRICA ABSTRACTA
aws_storage = AWSStorage(bucket='temperatures', max_retention=30)
aws_storage.create_bucket()
aws_storage.export_to_file('20150101', 21)

aws_db = AWSDatabase(db_name='weather_db', table_name='rain_probability')
aws_db.connect()
aws_db.insert_record(22)

print('\n************\n')

azure_storage = AzureStorage(bucket='temperatures', is_encrypted=False)
azure_storage.create_bucket()
azure_storage.export_to_file('20150101', 31)

azure_db = AzureDatabase(db_name='weather_db', table_name='rain_probability')
azure_db.connect()
azure_db.insert_record(32)

Creando bucket temperatures con retencion de 30 dias en **S3**
Exportando a archivo 20150101 en **S3** el contenido: 21
Conectando a la base de datos weather_db en **RDS**
Insertando el registro 22 en la tabla rain_probability de **RDS**

************

Creando bucket temperatures con encriptacion False en **Blobstorage**
Exportando a archivo 20150101 en **Blobstorage** el contenido: 31
Conectando a la base de datos weather_db en **Azure**
Insertando el registro 32 en la tabla rain_probability de **Azure**


In [36]:
#CLASE FABRICA ABSTRACTA
class AbstractFactoryCloud(ABC):

    @abstractmethod
    def create_storage(self, bucket) -> AbstractStorage:
        pass

    @abstractmethod
    def create_database(self, db_name, table_name) -> AbstractDatabase:
        pass

    def launch_database(self, record):        
        db = self.create_database(db_name='weather_db', table_name='rain_probability')
        db.connect()
        db.insert_record(record)

#FABRICAS CONCRETAS
class AWSFactory(AbstractFactoryCloud):

    def create_storage(self, bucket, max_retention) -> AbstractStorage:
        return AWSStorage(bucket=bucket, max_retention=max_retention)

    def create_database(self, db_name, table_name) -> AbstractDatabase:
        return AWSDatabase(db_name=db_name, table_name=table_name)
    
class AzureFactory(AbstractFactoryCloud):

    def create_storage(self, bucket, is_encrypted) -> AbstractStorage:
        return AzureStorage(bucket=bucket, is_encrypted=is_encrypted)
    
    def create_database(self, db_name, table_name) -> AbstractDatabase:
        return AzureDatabase(db_name=db_name, table_name=table_name)

In [38]:
#CON FABRICA ABSTRACTA
cloud_manager = AWSFactory()

storage_manager = cloud_manager.create_storage(bucket='temperatures', max_retention=30)
storage_manager.create_bucket()
storage_manager.export_to_file('20150101', 21)

cloud_manager.launch_database(record=22)

print('\n************\n')

cloud_manager = AzureFactory()

storage_manager = cloud_manager.create_storage(bucket='temperatures', is_encrypted=False)
storage_manager.create_bucket()
storage_manager.export_to_file('20150101', 21)

cloud_manager.launch_database(record=32)


Creando bucket temperatures con retencion de 30 dias en **S3**
Exportando a archivo 20150101 en **S3** el contenido: 21
Conectando a la base de datos weather_db en **RDS**
Insertando el registro 22 en la tabla rain_probability de **RDS**

************

Creando bucket temperatures con encriptacion False en **Blobstorage**
Exportando a archivo 20150101 en **Blobstorage** el contenido: 21
Conectando a la base de datos weather_db en **Azure**
Insertando el registro 32 en la tabla rain_probability de **Azure**


## BUILDER

In [51]:
#PRODUCT CLASSES
class Automobile:

    def __init__(self):
        self.color = None
        self.engine = None
        self.number_of_doors = None

    def describe_specifications(self):
        print('Color: {}'.format(self.color))
        print('Engine: {}'.format(self.engine))
        print('Number of doors: {}'.format(self.number_of_doors))

class Motobike:

    def __init__(self):
        self.color = None
        self.engine = None
        self.has_sidecar = None

    def describe_specifications(self):
        print('Color: {}'.format(self.color))
        print('Engine: {}'.format(self.engine))
        print('Has sidecar: {}'.format(self.has_sidecar))

#ABSTRACT VEHICLE BUILDER
class VehicleBuilder(ABC):

    @abstractmethod
    def set_color(self, color: str):
        pass

    @abstractmethod
    def set_engine(self, engine: str):
        pass

#CONCRETE AUTOMOBILE BUILDER
class AutomobileBuilder(VehicleBuilder):

    def __init__(self):
        self.automobile = Automobile()

    def set_color(self, color: str):
        self.automobile.color = color

    def set_engine(self, engine: str):
        self.automobile.engine = engine

    def set_number_of_doors(self, number_of_doors: int):
        self.automobile.number_of_doors = number_of_doors
    
#CONCRETE MOTOBIKE BUILDER
class MotobikeBuilder(VehicleBuilder):

    def __init__(self):
        self.motobike = Motobike()

    def set_color(self, color: str):
        self.motobike.color = color

    def set_engine(self, engine: str):
        self.motobike.engine = engine

    def set_sidecar(self, has_sidecar: bool):
        self.motobike.has_sidecar = has_sidecar
    
#DIRECTOR
class VehicleDirector:

    def __init__(self, builder: VehicleBuilder):
        self.builder = builder

    def construct_tsuru(self):
        self.builder.set_color('white')
        self.builder.set_engine('gasoline')
        self.builder.set_number_of_doors(4)

    def construct_harley(self):
        self.builder.set_color('black')
        self.builder.set_engine('gasoline')
        self.builder.set_sidecar(False)

    def construct_truck(self):
        self.builder.set_color('red')
        self.builder.set_engine('diesel')
        self.builder.set_number_of_doors(2)

    def construct_suzuki(self):
        self.builder.set_color('blue')
        self.builder.set_engine('gasoline')
        self.builder.set_sidecar(True)

In [50]:
#CLIENT CODE
auto_builder = AutomobileBuilder()
director = VehicleDirector(auto_builder)
director.construct_tsuru()
auto_builder.automobile.describe_specifications()

print('---')
director.construct_truck()
auto_builder.automobile.describe_specifications()

print('************')
moto_builder = MotobikeBuilder()
director = VehicleDirector(moto_builder)
director.construct_harley()
moto_builder.motobike.describe_specifications()

print('---')
director.construct_suzuki()
moto_builder.motobike.describe_specifications()

Color: white
Engine: gasoline
Number of doors: 4
---
Color: red
Engine: diesel
Number of doors: 2
************
Color: black
Engine: gasoline
Has sidecar: False
---
Color: blue
Engine: gasoline
Has sidecar: True


## PROTOTYPE

In [53]:
#CLASE PROTOTIPO ABSTRACTA
class PrototypeShape(ABC):

    @abstractmethod
    def clone(self):
        pass

    @abstractmethod
    def draw(self):
        pass

#CLASES PROTOTIPO CONCRETAS
class Circle(PrototypeShape):

    def __init__(self, color, radius):
        self.color = color
        self.radius = radius

    def clone(self):
        return Circle(self.color, self.radius)

    def draw(self):
        print(f'Drawing a {self.color} circle with radius {self.radius}')

class Square(PrototypeShape):

    def __init__(self, color, side_length):
        self.color = color
        self.side_length = side_length

    def clone(self):
        return Square(self.color, self.side_length)
    
    def draw(self):
        print(f'Drawing a {self.color} square with side length {self.side_length}')

In [55]:
#CIRCULOS CLONADOS
circle_prototype = Circle('red', 5)

circle_01 = circle_prototype.clone()
circle_01.radius = 10

circle_02 = circle_prototype.clone()
circle_02.color = 'blue'

circle_prototype.draw()
circle_01.draw()
circle_02.draw()

print('**************************')

#CUADRADOS CLONADOS
square_prototype = Square('green', 4)

square_01 = square_prototype.clone()
square_01.side_length = 8

square_02 = square_prototype.clone()
square_02.color = 'yellow'

square_prototype.draw()
square_01.draw()
square_02.draw()

Drawing a red circle with radius 5
Drawing a red circle with radius 10
Drawing a blue circle with radius 5
**************************
Drawing a green square with side length 4
Drawing a green square with side length 8
Drawing a yellow square with side length 4


## SINGLETON

In [56]:
#CLASE SINGLETON
class Singleton(object):
   
   legend = None

   def __new__(cls):

       if not hasattr(cls, 'instance'):

           cls.instance = super(Singleton, cls).__new__(cls)

       return cls.instance
   
   def set_legend(legend):
       self.legend = legend

In [58]:
obj_01 = Singleton()
obj_01.legend = 'The only one'
obj_02 = Singleton()

print(obj_01.legend)
print(obj_02.legend)

obj_02.legend = 'Changed legend'
print(obj_01.legend)
print(obj_02.legend)
print(obj_01 is obj_02)

The only one
The only one
Changed legend
Changed legend
True
